## Įterpimai

Ankstesniame pavyzdyje dirbome su aukštos dimensijos žodžių maišo vektoriais, kurių ilgis yra `vocab_size`, ir aiškiai konvertavome iš žemos dimensijos pozicinių reprezentacijų vektorių į retą vieno elemento reprezentaciją. Ši vieno elemento reprezentacija nėra efektyvi atminties požiūriu, be to, kiekvienas žodis yra traktuojamas nepriklausomai nuo kitų, t. y. vieno elemento užkoduoti vektoriai neišreiškia jokio semantinio panašumo tarp žodžių.

Šiame skyriuje toliau nagrinėsime **News AG** duomenų rinkinį. Pradėkime įkeldami duomenis ir pasinaudodami kai kuriomis ankstesnio užrašų knygelės apibrėžtimis.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## Kas yra įterpimas?

Įterpimo (**embedding**) idėja yra atvaizduoti žodžius mažesnės dimensijos tankiais vektoriais, kurie tam tikru būdu atspindi žodžio semantinę reikšmę. Vėliau aptarsime, kaip sukurti prasmingus žodžių įterpimus, tačiau šiuo metu tiesiog galvokime apie įterpimus kaip apie būdą sumažinti žodžio vektoriaus dimensiją.

Taigi, įterpimo sluoksnis priims žodį kaip įvestį ir pateiks išvesties vektorių su nurodytu `embedding_size`. Tam tikra prasme, tai labai panašu į `Linear` sluoksnį, tačiau vietoj vieno karšto kodavimo vektoriaus jis galės priimti žodžio numerį kaip įvestį.

Naudodami įterpimo sluoksnį kaip pirmąjį sluoksnį mūsų tinkle, galime pereiti nuo žodžių maišo (bag-of-words) prie **įterpimo maišo** (embedding bag) modelio, kuriame pirmiausia kiekvieną žodį mūsų tekste paverčiame atitinkamu įterpimu, o tada apskaičiuojame tam tikrą agregavimo funkciją visiems tiems įterpimams, pvz., `sum`, `average` arba `max`.

![Vaizdas, rodantis įterpimo klasifikatorių penkiems sekos žodžiams.](../../../../../lessons/5-NLP/14-Embeddings/images/embedding-classifier-example.png)

Mūsų klasifikatoriaus neuroninis tinklas prasidės įterpimo sluoksniu, tada agregavimo sluoksniu ir lineariu klasifikatoriumi viršuje:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### Darbas su kintamu sekos dydžiu

Dėl šios architektūros mūsų tinklui reikės sukurti minibatch'us tam tikru būdu. Ankstesniame skyriuje, naudojant žodžių maišo (BoW) metodą, visi BoW tensoriai minibatch'e turėjo vienodą dydį `vocab_size`, nepaisant tikrojo mūsų teksto sekos ilgio. Kai pereiname prie žodžių įterpimų (word embeddings), kiekviename teksto pavyzdyje turėsime skirtingą žodžių skaičių, o jungiant šiuos pavyzdžius į minibatch'us reikės taikyti tam tikrą užpildymą (padding).

Tai galima padaryti naudojant tą pačią techniką, pateikiant `collate_fn` funkciją duomenų šaltiniui:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### Mokymas įterpimo klasifikatoriaus

Dabar, kai apibrėžėme tinkamą duomenų įkroviklį, galime treniruoti modelį naudodami mokymo funkciją, kurią apibrėžėme ankstesniame skyriuje:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **Pastaba**: Čia mes treniruojame tik 25 tūkst. įrašų (mažiau nei vieną pilną epochą) dėl laiko taupymo, tačiau galite tęsti treniravimą, parašyti funkciją treniruoti kelias epochas ir eksperimentuoti su mokymosi tempo parametru, kad pasiektumėte didesnį tikslumą. Turėtumėte sugebėti pasiekti apie 90% tikslumą.


### EmbeddingBag sluoksnis ir kintamo ilgio sekų reprezentacija

Ankstesnėje architektūroje reikėjo visas sekas užpildyti iki vienodo ilgio, kad jos tilptų į mini paketą. Tai nėra pats efektyviausias būdas reprezentuoti kintamo ilgio sekas – kitas požiūris būtų naudoti **poslinkio** vektorių, kuris saugotų visų sekų poslinkius viename dideliame vektoriuje.

![Vaizdas, rodantis poslinkio sekos reprezentaciją](../../../../../lessons/5-NLP/14-Embeddings/images/offset-sequence-representation.png)

> **Note**: Aukščiau pateiktame paveikslėlyje rodoma simbolių seka, tačiau mūsų pavyzdyje dirbame su žodžių sekų reprezentacija. Vis dėlto bendras principas, kaip sekas reprezentuoti naudojant poslinkio vektorių, išlieka tas pats.

Norėdami dirbti su poslinkio reprezentacija, naudojame [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html) sluoksnį. Jis panašus į `Embedding`, tačiau kaip įvestį naudoja turinio vektorių ir poslinkio vektorių. Be to, jis apima vidurkinimo sluoksnį, kuris gali būti `mean`, `sum` arba `max`.

Štai modifikuotas tinklas, kuris naudoja `EmbeddingBag`:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

Norėdami paruošti duomenų rinkinį mokymui, turime pateikti konversijos funkciją, kuri paruoš poslinkio vektorių:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

Atkreipkite dėmesį, kad, skirtingai nei visuose ankstesniuose pavyzdžiuose, mūsų tinklas dabar priima du parametrus: duomenų vektorių ir poslinkio vektorių, kurie yra skirtingo dydžio. Panašiai, mūsų duomenų įkroviklis taip pat pateikia mums 3 reikšmes vietoj 2: tiek teksto, tiek poslinkio vektoriai pateikiami kaip ypatybės. Todėl turime šiek tiek pakoreguoti savo mokymo funkciją, kad tai būtų tinkamai apdorota:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## Semantiniai įterpiniai: Word2Vec

Mūsų ankstesniame pavyzdyje modelio įterpimo sluoksnis išmoko susieti žodžius su vektorinėmis reprezentacijomis, tačiau ši reprezentacija neturėjo daug semantinės prasmės. Būtų naudinga išmokti tokią vektorinę reprezentaciją, kurioje panašūs žodžiai ar sinonimai atitiktų vektorius, esančius arti vienas kito pagal tam tikrą vektorinį atstumą (pvz., euklidinį atstumą).

Tam reikia iš anksto apmokyti mūsų įterpimo modelį naudojant didelę tekstų kolekciją specifiniu būdu. Vienas iš pirmųjų būdų mokyti semantinius įterpinius vadinamas [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Jis pagrįstas dviem pagrindinėmis architektūromis, kurios naudojamos žodžių paskirstytai reprezentacijai kurti:

 - **Nuolatinis maišo žodžių modelis** (CBoW) — šioje architektūroje modelis mokomas numatyti žodį iš aplinkinio konteksto. Turint ngramą $(W_{-2},W_{-1},W_0,W_1,W_2)$, modelio tikslas yra numatyti $W_0$ iš $(W_{-2},W_{-1},W_1,W_2)$.
 - **Nuolatinis skip-gram modelis** yra priešingas CBoW. Modelis naudoja aplinkinį kontekstinių žodžių langą, kad numatytų dabartinį žodį.

CBoW yra greitesnis, o skip-gram yra lėtesnis, tačiau geriau reprezentuoja retus žodžius.

![Vaizdas, rodantis tiek CBoW, tiek Skip-Gram algoritmus, skirtus žodžiams paversti vektoriais.](../../../../../lessons/5-NLP/14-Embeddings/images/example-algorithms-for-converting-words-to-vectors.png)

Norėdami eksperimentuoti su Word2Vec įterpimu, iš anksto apmokytu naudojant Google News duomenų rinkinį, galime naudoti **gensim** biblioteką. Žemiau pateikiame žodžius, labiausiai panašius į 'neural'.

> **Note:** Kai pirmą kartą kuriate žodžių vektorius, jų atsisiuntimas gali užtrukti!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Mes taip pat galime apskaičiuoti vektorių įterpimus iš žodžio, kurie bus naudojami klasifikavimo modelio mokymui (aiškumo dėlei rodome tik pirmąsias 20 vektoriaus komponentų):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Puikus dalykas apie semantinius įterpimus yra tai, kad galite manipuliuoti vektoriaus kodavimu, kad pakeistumėte semantiką. Pavyzdžiui, galime paprašyti surasti žodį, kurio vektorinė reprezentacija būtų kuo artimesnė žodžiams *karalius* ir *moteris*, ir kuo toliau nuo žodžio *vyras*:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Tiek CBoW, tiek Skip-Grams yra „prognozuojančios“ įterptys, nes jos atsižvelgia tik į vietinius kontekstus. Word2Vec nepasinaudoja globaliu kontekstu.

**FastText** remiasi Word2Vec, mokydamas vektorių reprezentacijas kiekvienam žodžiui ir simbolių n-gramas, esančias žodyje. Šių reprezentacijų reikšmės kiekviename mokymo žingsnyje yra vidurkinamos į vieną vektorių. Nors tai prideda daug papildomų skaičiavimų priešmokymio metu, tai leidžia žodžių įterptims koduoti subžodžių informaciją.

Kitas metodas, **GloVe**, pasinaudoja koegzistavimo matricos idėja, naudodamas neuroninius metodus, kad išskaidytų koegzistavimo matricą į išraiškingesnius ir nelinijinius žodžių vektorius.

Galite eksperimentuoti su pavyzdžiu, keisdami įterptis į FastText ir GloVe, nes gensim palaiko kelis skirtingus žodžių įterpimo modelius.


## Naudojant iš anksto apmokytus įterpimus PyTorch

Galime pakeisti aukščiau pateiktą pavyzdį, kad iš anksto užpildytume matricą mūsų įterpimo sluoksnyje semantiniais įterpimais, tokiais kaip Word2Vec. Turime atsižvelgti į tai, kad iš anksto apmokytų įterpimų ir mūsų teksto korpuso žodynai greičiausiai nesutaps, todėl trūkstamų žodžių svorius inicializuosime atsitiktinėmis reikšmėmis:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


Dabar treniruokime mūsų modelį. Atkreipkite dėmesį, kad modelio treniravimas užtrunka žymiai ilgiau nei ankstesniame pavyzdyje, dėl didesnio įterpimo sluoksnio dydžio ir daug didesnio parametrų skaičiaus. Taip pat dėl to gali prireikti treniruoti modelį su daugiau pavyzdžių, jei norime išvengti per didelio pritaikymo.


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

Mūsų atveju nematome didelio tikslumo padidėjimo, greičiausiai dėl labai skirtingų žodynų.  
Norint išspręsti skirtingų žodynų problemą, galime naudoti vieną iš šių sprendimų:  
* Iš naujo apmokyti word2vec modelį pagal mūsų žodyną  
* Įkelti mūsų duomenų rinkinį su žodynu iš iš anksto apmokyto word2vec modelio. Žodyną, naudojamą duomenų rinkiniui įkelti, galima nurodyti įkėlimo metu.  

Pastarasis metodas atrodo paprastesnis, ypač todėl, kad PyTorch `torchtext` sistema turi integruotą palaikymą įterpimams. Pavyzdžiui, galime sukurti žodyną, pagrįstą GloVe, tokiu būdu:  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


Įkeltas žodynas turi šias pagrindines operacijas:
* `vocab.stoi` žodynas leidžia mums konvertuoti žodį į jo indeksą žodyne
* `vocab.itos` atlieka priešingą veiksmą - konvertuoja skaičių į žodį
* `vocab.vectors` yra įterptųjų vektorių masyvas, todėl norint gauti žodžio `s` įterptį, turime naudoti `vocab.vectors[vocab.stoi[s]]`

Štai pavyzdys, kaip manipuliuoti įterptimis, kad būtų pademonstruota lygtis **kind-man+woman = queen** (turėjau šiek tiek pakoreguoti koeficientą, kad tai veiktų):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

Norint apmokyti klasifikatorių naudojant šiuos įterpimus, pirmiausia turime užkoduoti savo duomenų rinkinį naudodami GloVe žodyną:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

Kaip matėme aukščiau, visi vektorių įterpimai saugomi `vocab.vectors` matricoje. Tai labai palengvina šių svorių įkėlimą į įterpimo sluoksnio svorius naudojant paprastą kopijavimą:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

Viena iš priežasčių, kodėl nematome reikšmingo tikslumo padidėjimo, yra ta, kad kai kurių žodžių iš mūsų duomenų rinkinio nėra iš anksto apmokyto GloVe žodyno, todėl jie iš esmės ignoruojami. Norėdami įveikti šią problemą, galime apmokyti savo įterpimus pagal mūsų duomenų rinkinį.


## Kontekstinės įterptys

Viena pagrindinių tradicinių iš anksto apmokytų įterpčių, tokių kaip Word2Vec, apribojimų yra žodžių reikšmių išskyrimo problema. Nors iš anksto apmokytos įterptys gali užfiksuoti dalį žodžių reikšmės kontekste, visos galimos žodžio reikšmės yra užkoduojamos toje pačioje įterptyje. Tai gali sukelti problemų tolimesniuose modeliuose, nes daugelis žodžių, pavyzdžiui, žodis „play“, turi skirtingas reikšmes priklausomai nuo konteksto, kuriame jie naudojami.

Pavyzdžiui, žodis „play“ šiuose dviejuose sakiniuose turi gana skirtingas reikšmes:
- Aš nuėjau į **spektaklį** teatre.
- Jonas nori **žaisti** su savo draugais.

Aukščiau pateiktos iš anksto apmokytos įterptys abu šiuos žodžio „play“ reikšmes pateikia toje pačioje įterptyje. Norint įveikti šį apribojimą, reikia kurti įterptis, pagrįstas **kalbos modeliu**, kuris yra apmokytas naudojant didelį tekstų korpusą ir *žino*, kaip žodžiai gali būti naudojami skirtinguose kontekstuose. Kontekstinių įterpčių aptarimas nėra šio vadovo dalis, tačiau mes prie jų sugrįšime, kai kalbėsime apie kalbos modelius kitame skyriuje.



---

**Atsakomybės apribojimas**:  
Šis dokumentas buvo išverstas naudojant AI vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, prašome atkreipti dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Kritinei informacijai rekomenduojama naudoti profesionalų žmogaus vertimą. Mes neprisiimame atsakomybės už nesusipratimus ar neteisingą interpretaciją, atsiradusią dėl šio vertimo naudojimo.
